# 6.1. Obtención de datos financieros

In [ ]:
import pandas as pd
import requests

Vamos a conectarnos con un servicio web para obtener datos financieros

1. Regístrate en iexcloud: https://iexcloud.io/

IEX es la web de un agregador en donde podemos encontrar infinidad de datos

Si hacemos un "uso intensivo", hay que pagar. Pero también tiene una serie de llamadas al mes, que son gratuítas. 

Para lo que vamos a hacer en el máster, es más que perfecto

<center>
<img src="./imgs/portada_iex.png"  alt="drawing" width="1000"/>
</center>

A la hora de registrarte, hay una opción que es 9,99 al mes y otra que es gratuita. Elije esta última.

<center>
<img src="./imgs/gratis.png"  alt="drawing" width="400"/>
</center>

Una vez estéis registrados, tenéis que estar en una pantalla como esta

<center>
<img src="./imgs/registrado.png"  alt="drawing" width="1000"/>
</center>

2. Genera y copia un token de acceso.

En el menú lateral, ve a la llave (access & security) 

<center>
<img src="./imgs/acceess.png"  alt="drawing" width="800"/>
</center>

El token es "la contraseña" que os dará acceso a la descarga de datos

En cada petición que hagamos, deberemos incluir el token

Si no tenéis uno ya creado por defecto, cread uno

3. Observa la documentación, mira los datos que están disponibles.

En la interrogación de arriba a la derecha, seleccionad API Docs

<center>
<img src="./imgs/api_doc.png"  alt="drawing" width="900"/>
</center>

Un API es una dirección web. De hecho, primero haremos los ejemplos en el navegador, para después hacerlos desde python.

Vamos a construir una dirección web, y vamos a añadir el token, para hacer una petición, que nos devolverá un json

La dirección a la que tenemos que llamar es la subrayada

<center>
<img src="./imgs/direccion.png"  alt="drawing" width="400"/>
</center>

Por ejemplo, en historical prices podemos obtener los datos históricos de cualquier stock

<center>
<img src="./imgs/historical.png"  alt="drawing" width="900"/>
</center>

En todas las páginas, más abajo, podemos encontrar ejemplos sobre cómo hacer la llamada

Si pinchamos sobre ellos, nos aparecerá una web indicando que el API KEY es necesaria

Si queremos hacer la llamada, directamente sobre la web, debemos añadir nuestro token, poniendo al final de la dirección una interrogación + nuestro token

<center>
<img src="./imgs/direccion_token.png"  alt="drawing" width="900"/>
</center>

Con el token incluido, podemos ver cómo nos ha traído todos los datos

En Firefox se ve como en la imagen, en Chrome se ve menos bonito.

<center>
<img src="./imgs/datos_web.png"  alt="drawing" width="400"/>
</center>

4. Explora y encuenta los endpoints con los que se obtienen las bolsas disponibles.



<center>
<img src="./imgs/reference_Data.png"  alt="drawing" width="200"/>
</center>

En reference data podrás encontrar International Exchanges

<center>
<img src="./imgs/international_exchanges.png"  alt="drawing" width="400"/>
</center>

5. Encuenta los endpoints con los que se obtienen los símbolos de los activos que cotizan en una bolsa

<center>
<img src="./imgs/international_symbols.png"  alt="drawing" width="600"/>
</center>

Dedicad unos minutos a explorar la documentación y ver distintos ejemplos

6. Prueba a obtener los datos de los últimos dos apartados desde el navegador. Hazlo, por ejemplo, para Bolsa de Madrid

Para ello contruye una url de la siguiente manera: 

```https://cloud.iexapis.com/stable/------rellena lo que quieras pedir ----?token=TU_TOKEN```

Ojo. Solo tenemos 50.000 datos gratuitos al mes: una llamada web también consume datos

En la consola de inicio podéis ver cuantos datos lleváis consumidos

<center>
<img src="./imgs/uso.png"  alt="drawing" width="600"/>
</center>

Para las bolsas internacionales necesitamos /ref-data/exchanges

Bolsa de Madrid es XMAD

<center>
<img src="./imgs/bolsas_internacionales.png"  alt="drawing" width="900"/>
</center>

<center>
<img src="./imgs/bolsa_de_madrid.png"  alt="drawing" width="400"/>
</center>

Para los símbolos internacionales necesitamos /ref-data/exchange/{exchange}/symbols

Las llaves no hay que ponerlas. Hay que poner el símbolo del exchange de Madrid: XMAD

<center>
<img src="./imgs/activos_madrid.png"  alt="drawing" width="1000"/>
</center>

7. Ahora pide lo mismo desde python usando la libreria requests de la siguiente forma:

```python
import requests
url = f'https://cloud.iexapis.com/stable/------rellena lo que quieras pedir ----'
r = requests.get(url, params={'token': TU_TOKEN})
exchanges = pd.read_json(r.content)
```

Obtenemos las bolsas internacionales

In [ ]:
TOKEN = 'TU_TOKEN'
base_url = 'https://cloud.iexapis.com/stable' 

In [ ]:
url = f'{base_url}/ref-data/exchanges'
r = requests.get(url, params={'token':TOKEN})
exchanges = pd.DataFrame(r.json())

In [ ]:
exchanges.head()

Obtenemos los activos que cotizan en Bolsa de Madrid

In [ ]:
exchange = 'XMAD'
url = f"{base_url}/ref-data/exchange/{exchange}/symbols"
r = requests.get(url, params={'token': TOKEN})
symbols = pd.DataFrame(r.json())

In [ ]:
symbols.shape

In [ ]:
symbols.head()

8. Crea una función que retorne el open, high, low, close y volumen diario del último año, dado un ticker.

Consulta la sección y ejemplos de historical prices

Utiliza el ticker de Nvidia para invocar la función: 'nvda'

In [ ]:
def get_data_iex(symbol, st_range='1y'):
    url = f"{base_url}/stock/{symbol}/chart/{st_range}"
    r = requests.get(url, params={'token': TOKEN})
    data = pd.DataFrame(r.json())
    data = data.set_index('date')
    data.index = pd.to_datetime(data.index)
    data.name = symbol
    return data

In [ ]:
data_day = get_data_iex('nvda')
data_day.head()

In [ ]:
data_day_OLHC = data_day.loc[:, 'close':'open']
data_day_OLHC.head()

9. Crea una función que obtenga los datos, minuto a minuto, de un día completo, dado un ticker.

Utiliza, por ejemplo, el ticker de Apple: 'aapl'

In [ ]:
def datos_intradia(symbol, rango_temporal='minute'):    
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/intraday-prices'
    r = requests.get(url, params={'token': TOKEN})
    data = pd.DataFrame(r.json())    
    data['time'] = pd.to_datetime(data['date'] + ' ' + data['minute'])
    data = data.drop(['date', rango_temporal], axis=1)
    data = data.set_index('time')  
    return data

In [ ]:
data_intraday = datos_intradia(symbol="aapl", rango_temporal='minute')
data_intraday.head()

In [ ]:
data_intraday_OLHC = data_intraday.loc[:, 'high':'close']
data_intraday_OLHC.head()

10. Pinta las figuras de velas correspondientes a los dos ejercicios anteriores

In [ ]:
import mplfinance as mpf

In [ ]:
mpf.plot(data_day_OLHC, type='candle')

In [ ]:
mpf.plot(data_intraday_OLHC, type='candle')

## Otros

### Investpy

Para descargar datos de Investing

investpy: https://investpy.readthedocs.io/

<center>
<img src="./imgs/investing.png"  alt="drawing" width="1000"/>
</center>

Ojo. Lo que está haciendo esta librería es un webscraping. Por lo que si cambia la web, la fiabilidad de los datos sería muy cuestionable.

### Pandas Data Reader

Pandas tiene una librería para descargar datos de multitud de fuentes

pandas-datareader: https://pandas-datareader.readthedocs.io

<center>
<img src="./imgs/pandas_datareader.png"  alt="drawing" width="1000"/>
</center>

Esta librería nos permite conectarnos con multitud de fuentes de datos: IEX, Alpha Vantage, Quandl...

<center>
<img src="./imgs/fuentes_pandas.png"  alt="drawing" width="1000"/>
</center>

Por ejemplo, para conectarnos a IEX desde Pandas Data Reader

<center>
<img src="./imgs/iex_pandas.png"  alt="drawing" width="1000"/>
</center>